In [3]:
import os
import pandas as pd
import csv

# Function to detect delimiter by reading the first line of the file
def detect_delimiter(txt_file_path):
    with open(txt_file_path, 'r') as file:
        first_line = file.readline()
        # Check common delimiters: comma, tab, and space
        if ',' in first_line:
            return ','
        elif '\t' in first_line:
            return '\t'
        elif ' ' in first_line:
            return ' '
        else:
            return None

# Function to convert a single .txt file to .xlsx, including header information
def convert_txt_to_xlsx(txt_file_path, output_folder):
    try:
        # Read the entire file to handle both header and data
        with open(txt_file_path, 'r') as file:
            # Assuming the header is the first few lines before the data table
            header_lines = []
            for i in range(5):  # Adjust the number of header lines accordingly
                line = file.readline().strip()
                header_lines.append(line)
        
        # Detect the delimiter
        delimiter = detect_delimiter(txt_file_path)
        if not delimiter:
            raise ValueError(f"Unknown delimiter in file: {txt_file_path}")

        # Read the actual data, skipping the header lines
        df = pd.read_csv(txt_file_path, delimiter=delimiter, skiprows=len(header_lines), engine="python", quoting=csv.QUOTE_NONE)
        
        # Prepare Excel writer
        base_name = os.path.basename(txt_file_path).replace('.txt', '.xlsx')  # Changed to .xlsx
        xlsx_file_path = os.path.join(output_folder, base_name)
        
        with pd.ExcelWriter(xlsx_file_path, engine='openpyxl') as writer:
            # Write header lines as text at the top of the Excel sheet
            header_df = pd.DataFrame(header_lines)
            header_df.to_excel(writer, sheet_name='Sheet1', index=False, header=False)

            # Append the main data underneath the header
            df.to_excel(writer, sheet_name='Sheet1', index=False, startrow=len(header_lines) + 2)
        
        print(f"Converted: {txt_file_path} to {xlsx_file_path}")

    except pd.errors.ParserError as e:
        print(f"ParserError while reading {txt_file_path}: {e}")
    except Exception as e:
        print(f"Error while converting {txt_file_path}: {e}")

# Main function to convert all .txt files in a folder to .xlsx
def batch_convert_txt_to_xlsx(input_folder, output_folder):
    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    # Loop through all the files in the input folder
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.txt'):
            txt_file_path = os.path.join(input_folder, file_name)
            convert_txt_to_xlsx(txt_file_path, output_folder)

# Example usage:
input_folder = "/Users/brenthokeness/Desktop/BiomechData/ALEXUS_TRIALS/alexustrials_TXT"  # Replace with your folder path containing .txt files
output_folder = "/Users/brenthokeness/Desktop/BiomechData/ALEXUS_TRIALS/alexustrials_XLSL"  # Replace with the folder path where .xlsx files will be saved

batch_convert_txt_to_xlsx(input_folder, output_folder)

Converted: /Users/brenthokeness/Desktop/BiomechData/ALEXUS_TRIALS/alexustrials_TXT/JNH_2-18_VJ2_Post - HotVsCOld.txt to /Users/brenthokeness/Desktop/BiomechData/ALEXUS_TRIALS/alexustrials_XLSL/JNH_2-18_VJ2_Post - HotVsCOld.xlsx
Converted: /Users/brenthokeness/Desktop/BiomechData/ALEXUS_TRIALS/alexustrials_TXT/PBJ_2-26_VJ2_Post - HotVsCOld.txt to /Users/brenthokeness/Desktop/BiomechData/ALEXUS_TRIALS/alexustrials_XLSL/PBJ_2-26_VJ2_Post - HotVsCOld.xlsx
Converted: /Users/brenthokeness/Desktop/BiomechData/ALEXUS_TRIALS/alexustrials_TXT/DEP_2-25_VJ1_Pre - HotVsCOld.txt to /Users/brenthokeness/Desktop/BiomechData/ALEXUS_TRIALS/alexustrials_XLSL/DEP_2-25_VJ1_Pre - HotVsCOld.xlsx
Converted: /Users/brenthokeness/Desktop/BiomechData/ALEXUS_TRIALS/alexustrials_TXT/BFL_2-26_VJ2_Post - HotVsCOld.txt to /Users/brenthokeness/Desktop/BiomechData/ALEXUS_TRIALS/alexustrials_XLSL/BFL_2-26_VJ2_Post - HotVsCOld.xlsx
Converted: /Users/brenthokeness/Desktop/BiomechData/ALEXUS_TRIALS/alexustrials_TXT/DEP_2-2

In [8]:
import pandas as pd
import os
import glob

# Define paths
excel_folder = "/Users/brenthokeness/Desktop/BiomechData/ALEXUS_TRIALS/alexustrials_XLSL"
output_file = "/Users/brenthokeness/Desktop/BiomechData/ALEXUS_TRIALS/ALEXUS_analysis_results.xlsx"

# Columns to analyze
columns_to_analyze = [
    "R_GRF_Mag",
    "L_GRF_Mag",
    "Left_Ankle_FlexExt",
    "Right_Ankle_FlexExt",
    "L_GRF_Mag_Norm",
    "R_GRF_Mag_Norm",
    "Total_GRF_Vertical_Norm",
    "Total_GRF_Vertical",
    "COM_Pos",
    "Max_Height",
    "MinHeight",
    "PeakJumpHeight",
    "Work",
    "COM_Velocity",
    "weight",
    "Height",
    "GRF__Diff",
    "Total_GRF_Mag_Norm",
    "Left_AnkleForce_Mag_Norm",
    "Right_AnkleForce_Mag_Norm",
]

# Create empty lists to store results
results = []

# Process each Excel file
for excel_file in glob.glob(os.path.join(excel_folder, "*.xlsx")):
    try:
        # Read the Excel file, skipping to the header row
        df = pd.read_excel(excel_file, header=7)
        
        # Find indices where VEM1 and VEM2 are 1
        vem1_index = df[df['VEM1'] == 1].index[0]
        vem2_index = df[df['VEM2'] == 1].index[0]
        
        # Get the range of data between (and including) these indices
        data_range = df.iloc[vem1_index:vem2_index + 1]
        
        # Calculate statistics for each column
        file_results = {
            'File_Name': os.path.basename(excel_file)
        }
        
        for column in columns_to_analyze:
            file_results[f'{column}_Average'] = data_range[column].mean()
            file_results[f'{column}_Max'] = data_range[column].max()
        
        results.append(file_results)
        print(f"Processed: {os.path.basename(excel_file)}")
        
    except Exception as e:
        print(f"Error processing {excel_file}: {e}")

# Create DataFrame from results and save to Excel
results_df = pd.DataFrame(results)

# Reorder columns to group statistics by measurement type
column_order = ['File_Name']
for col in columns_to_analyze:
    column_order.extend([f'{col}_Average', f'{col}_Max'])

results_df = results_df[column_order]

# Save to Excel
results_df.to_excel(output_file, index=False)
print(f"\nResults saved to: {output_file}")

Processed: BFL_2-26_VJ1_Pre - HotVsCOld.xlsx
Processed: GKG_2-24_VJ1_Post - HotVsCOld.xlsx
Processed: EAD_2-20_VJ1_Pre - HotVsCOld.xlsx
Processed: JNH_2-18_VJ1_Pre - HotVsCOld.xlsx
Processed: PBJ_2-26_VJ1_Pre - HotVsCOld.xlsx
Processed: EAD_2-20_VJ1_Post - HotVsCOld.xlsx
Processed: SRT_2-20_VJ2_Post - HotVsCOld.xlsx
Processed: DEP_2-25_VJ2_Post - HotVsCOld.xlsx
Processed: ZMB_2-19_VJ2_Post - HotVsCOld.xlsx
Processed: SRT_2-20_VJ1_Pre - HotVsCOld.xlsx
Processed: NJH_2-27_VJ1_Post - HotVsCOld.xlsx
Processed: MVL_2-25_VJ1_Pre - HotVsCOld.xlsx
Processed: MKH_2-27_VJ2_Post - HotVsCOld.xlsx
Processed: HSA_2-27_VJ2_Post - HotVsCOld.xlsx
Processed: DJI_2-27_VJ2_Post - HotVsCOld.xlsx
Error processing /Users/brenthokeness/Desktop/BiomechData/ALEXUS_TRIALS/alexustrials_XLSL/CEV_2-13_VJ2_Post - HotVsCOld.xlsx: index 0 is out of bounds for axis 0 with size 0
Error processing /Users/brenthokeness/Desktop/BiomechData/ALEXUS_TRIALS/alexustrials_XLSL/~$BFL_2-26_VJ1_Pre - HotVsCOld.xlsx: Excel file form